In [2]:
pip install pyvis

Note: you may need to restart the kernel to use updated packages.


In [3]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('Arquivos/'):
    f.extend(filenames)
    break

In [4]:
import pandas as pd
dataframe_interacao = pd.DataFrame()

for cont, arquivo in enumerate(f):
    frame = pd.read_excel('Arquivos/' + arquivo)

    dataframe_interacao = dataframe_interacao.append(
        frame, 
        ignore_index = True
    )

C:\Users\Pedro\AppData\Local\Temp\ipykernel_17124\1092503128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_interacao = dataframe_interacao.append(
C:\Users\Pedro\AppData\Local\Temp\ipykernel_17124\1092503128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_interacao = dataframe_interacao.append(
C:\Users\Pedro\AppData\Local\Temp\ipykernel_17124\1092503128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_interacao = dataframe_interacao.append(
C:\Users\Pedro\AppData\Local\Temp\ipykernel_17124\1092503128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_interacao = dataframe_interacao.

In [5]:
dataframe_interacao.head(50)

,Unnamed: 0,ID_ISSUE,LINK,AUTOR,DATA_ABERTURA,DATA_FECHAMENTO,USUARIO_COMENTARIO
0,0,614602330,https://github.com/nodejs/node/issues/33306,6070939,08/05/2020,08/05/2020,54468
1,1,614602330,https://github.com/nodejs/node/issues/33306,6070939,08/05/2020,08/05/2020,275871
2,2,614598999,https://github.com/nodejs/node/issues/33305,2234073,08/05/2020,08/05/2020,498775
3,3,614598999,https://github.com/nodejs/node/issues/33305,2234073,08/05/2020,08/05/2020,498775
4,4,614598999,https://github.com/nodejs/node/issues/33305,2234073,08/05/2020,08/05/2020,2234073
5,5,614567447,https://github.com/nodejs/node/issues/33303,403268,08/05/2020,Não Concluída,17770407
6,6,614559027,https://github.com/nodejs/node/issues/33302,201101,08/05/2020,12/05/2020,275871
7,7,614559027,https://github.com/nodejs/node/issues/33302,201101,08/05/2020,12/05/2020,201101
8,8,614559027,https://github.com/nodejs/node/issues/33302,201101,08/05/2020,12/05/2020,275871
9,9,614559027,https://github.com/nodejs/node/issues/33302,201101,08/05/2020,12/05/2020,201101


In [27]:
from datetime import *

data_inicial = '01/01/2018'
data_final = '01/01/2019'

data_inicial = datetime.strptime(
    data_inicial, 
    '%d/%m/%Y'
)

data_final = datetime.strptime(
    data_final, 
    '%d/%m/%Y'
)


In [28]:
relacoes = dict()

vertices = list()
arestas = list()

for id_issue in dataframe_interacao['ID_ISSUE'].unique():
    data_inicial_issue = dataframe_interacao[dataframe_interacao['ID_ISSUE'] == id_issue].values[0][4]
    data_final_issue = dataframe_interacao[dataframe_interacao['ID_ISSUE'] == id_issue].values[0][5]

    data_inicial_issue = datetime.strptime(
        data_inicial_issue, 
        '%d/%m/%Y'
    )

    data_incompleta = False

    if data_final_issue == 'Não Concluída':
        data_incompleta = True
    else:
        data_final_issue = datetime.strptime(
            data_final_issue, 
            '%d/%m/%Y'
        )
        
    
    if data_incompleta == True:
        pass
    elif data_inicial_issue < data_inicial:
        pass
    elif data_final_issue > data_final:
        pass
    else: 
        comentarios = dataframe_interacao[dataframe_interacao['ID_ISSUE'] == id_issue]
        
        usuarios_anteriores = []
        for indice, comentario in enumerate(comentarios.values):
            if indice == 0:
                usuarios_anteriores.append(comentario[3])
                
                if comentario[3] not in vertices:
                    vertices.append(comentario[3])
            
            for usuario_anterior in usuarios_anteriores:
                if comentario[6] not in vertices:
                    vertices.append(comentario[6])
                    
                if usuario_anterior > comentario[6]:
                    chave = '{}_{}'.format(usuario_anterior, comentario[6])

                    if chave not in relacoes:
                        relacoes[chave] = 1
                    else:
                        relacoes[chave] = relacoes[chave] + 1
                    
                elif usuario_anterior < comentario[6]:
                    chave = '{}_{}'.format(comentario[6], usuario_anterior)

                    if chave not in relacoes:
                        relacoes[chave] = 1
                    else:
                        relacoes[chave] = relacoes[chave] + 1
                
                if comentario[6] not in usuarios_anteriores:
                    usuarios_anteriores.append(comentario[6])

In [29]:
list(relacoes.items())

[('4402631_54666', 1),
 ('31628630_54666', 8),
 ('31628630_988441', 6),
 ('988441_54666', 3),
 ('3171327_54666', 3),
 ('39114273_10393198', 3),
 ('39114273_23313266', 2),
 ('23313266_10393198', 2),
 ('39114273_899444', 2),
 ('10393198_899444', 28),
 ('23313266_899444', 2),
 ('694376_275871', 3),
 ('694376_45469', 1),
 ('275871_45469', 3),
 ('23313266_10137', 2),
 ('718899_10137', 17),
 ('23313266_718899', 5),
 ('10428082_718899', 3),
 ('17013303_3109072', 1),
 ('13121330_3109072', 26),
 ('17013303_13121330', 1),
 ('3109072_2512748', 3),
 ('17013303_2512748', 1),
 ('13121330_2512748', 4),
 ('6639188_5952481', 3),
 ('6639188_718899', 3),
 ('5952481_718899', 116),
 ('6639188_6599179', 3),
 ('6599179_5952481', 2),
 ('6599179_718899', 2),
 ('1134733_54666', 4),
 ('1134733_718899', 3),
 ('718899_54666', 62),
 ('13121330_131406', 2),
 ('718899_131406', 4),
 ('13121330_718899', 49),
 ('490531_54666', 6),
 ('718899_490531', 5),
 ('498775_490531', 5),
 ('498775_54666', 2),
 ('718899_498775', 48)

In [30]:
from pyvis.network import Network
net = Network('720px', '1080px')
net.barnes_hut()

for usuario in vertices:
    net.add_node(usuario)

for relacao in list(relacoes.items()):
    usuarios = relacao[0].split('_')
    peso = relacao[1]

    usuarioA = int(usuarios[0])
    usuarioB = int(usuarios[1])

    net.add_edge(usuarioA, usuarioB, width = peso)

In [ ]:
net.show('Grafos/Grafo2018-2019.html')